<b>Media Wiki API </b>

2. Now grab data for the 5 articles you chose from the Fandom.com wiki you identified and grab revision/edit data from there. (Hint: Your wikipedia work will give you lots of clues here: for example, the fandom API endpoint for The Wire is https://thewire.fandom.com/api.php and the Fandom API, as I said in class, is the same as the Wikipedia API). Produce answers to the same three questions (i, ii, and iii) above but using this dataset.

In [67]:
#(i) what proportion of those edits were made by users without accounts ("anon")
#(ii) what proportion of those edits were marked as "minor"

import requests
import json

In [68]:
dict1 = {"website": "adventuretime.fandom.com", "page_title": "Finn_the_Human"}
dict2 = {"website": "adventuretime.fandom.com", "page_title": "Fionna_and_Cake"}
dict3 = {"website": "adventuretime.fandom.com", "page_title": "Jake_the_Dog"}
dict4 = {"website": "adventuretime.fandom.com", "page_title": "Marceline_the_Vampire_Queen"}
dict5 = {"website": "adventuretime.fandom.com", "page_title": "Princess_Bubblegum"}

dict_list = [dict1, dict2, dict3, dict4, dict5]

with open("FD_AdventureTimeCharacters.jsonl", "w") as adventure_time_characters:
    json.dump(dict1, adventure_time_characters)
    adventure_time_characters.write('\n')
    json.dump(dict2, adventure_time_characters)
    adventure_time_characters.write('\n')
    json.dump(dict3, adventure_time_characters)
    adventure_time_characters.write('\n')
    json.dump(dict4, adventure_time_characters)
    adventure_time_characters.write('\n')
    json.dump(dict5, adventure_time_characters)

In [69]:
#(i) what proportion of those edits were made by users without accounts ("anon")
#(ii) what proportion of those edits were marked as "minor"
def get_article_revision_json(title):
    api_answers = []
    fd_api_url = "https://adventuretime.fandom.com/api.php/"
    parameters = {'action' : 'query',
                  'titles' : title,
                  'prop' : 'revisions',
                  'rvprop' : 'flags|timestamp|user|size|ids',
                  'rvlimit' : 500,
                  'format' : 'json',
                   }
    while True:
        call = requests.get(fd_api_url, params=parameters)
        api_answer = call.json()
        api_answers.append(api_answer)
        if 'continue' in api_answer.keys():
            parameters.update(api_answer['continue'])
        else:
            break
        
    return(api_answers)

In [70]:
with open('FD_AdventureTimeCharacters.jsonl', 'r') as input_file,\
    open("FD_Adventure_Time_revisions.jsonl", 'w') as output_file:
    
    for line in input_file.readlines():
        line_dict = json.loads(line)
        page_title = line_dict["page_title"]
        
        print(f"now working on: {page_title}")
        api_answers = get_article_revision_json(page_title)
        for api_answer in api_answers:
            print(json.dumps(api_answer), file=output_file)

now working on: Finn_the_Human
now working on: Fionna_and_Cake
now working on: Jake_the_Dog
now working on: Marceline_the_Vampire_Queen
now working on: Princess_Bubblegum


In [71]:
revisions = []
minor_revisions_count = 0
for page_id in pages.keys():
    if "revisions" not in pages[page_id]:
        continue
    
    query_revisions = pages[page_id]["revisions"]
    title = pages[page_id]['title']

In [72]:
with open("FD_Adventure_Time_revisions.jsonl", 'r') as input_file:
    for line in input_file.readlines():
        api_answer = json.loads(line)

        # Get the second dictionary in the "query" key
        pages = api_answer["query"]["pages"]
        second_dict = None
        for page_id in pages.keys():
            if page_id != "-1":
                second_dict = pages[page_id]
                break
        
        # If the second dictionary exists, extract the revisions and title
        if second_dict is not None and "revisions" in second_dict:
            query_revisions = second_dict["revisions"]
            title = second_dict['title']
            
            # Iterate over each revision and process it as before
            for rev in query_revisions:
                if "userhidden" in rev.keys():
                    continue

                rev["title"] = title

                if "anon" in rev.keys():
                    rev["anon"] = True
                else:
                    rev["anon"] = False

                if "minor" in rev.keys():
                    rev["minor"] = True
                    minor_revisions_count += 1
                else:
                    rev["minor"] = False

                rev["timestamp"] = rev["timestamp"].replace("T", " ")
                rev["timestamp"] = rev["timestamp"].replace("Z", "")

                revisions.append(rev)
num_edits = len(revisions)
num_anon = 0

for rev in revisions:
    if rev["anon"]:
        num_anon = num_anon + 1

prop_anon = num_anon / num_edits
prop_minor = minor_revisions_count / num_edits

print(f"proportion anon: {prop_anon}")
print(f"proportion minor: {prop_minor}")

proportion anon: 0.24601959583588487
proportion minor: 0.1299755052051439


In [73]:
#(iii) make and share a visualization of the total number of edits across those 5 articles over time 
#(I didn't do this in class but I made the TSV file would allow this).

edits_by_day = {}
for rev in revisions:
    day_string = rev['timestamp'][0:10]

    if day_string in edits_by_day.keys():
        edits_by_day[day_string] = edits_by_day[day_string] + 1
    else:
        edits_by_day[day_string] = 1

In [74]:
edits_by_day = {}

for rev in revisions:
    day_string = rev['timestamp'][0:10]
    title = rev['title']
    key = (day_string, title)

    if key in edits_by_day.keys():
        edits_by_day[key] += 1
    else:
        edits_by_day[key] = 1

In [75]:
edits_by_day

{('2023-04-16', 'Fionna and Cake'): 1,
 ('2022-08-13', 'Fionna and Cake'): 1,
 ('2022-01-24', 'Fionna and Cake'): 1,
 ('2022-01-23', 'Fionna and Cake'): 1,
 ('2021-09-29', 'Fionna and Cake'): 2,
 ('2021-08-28', 'Fionna and Cake'): 1,
 ('2021-08-19', 'Fionna and Cake'): 2,
 ('2021-06-21', 'Fionna and Cake'): 1,
 ('2021-06-14', 'Fionna and Cake'): 1,
 ('2021-03-16', 'Fionna and Cake'): 2,
 ('2021-02-25', 'Fionna and Cake'): 1,
 ('2021-02-23', 'Fionna and Cake'): 1,
 ('2021-02-21', 'Fionna and Cake'): 4,
 ('2021-02-20', 'Fionna and Cake'): 1,
 ('2021-01-27', 'Fionna and Cake'): 1,
 ('2020-09-06', 'Fionna and Cake'): 1,
 ('2020-07-15', 'Fionna and Cake'): 1,
 ('2019-12-28', 'Fionna and Cake'): 1,
 ('2019-03-22', 'Fionna and Cake'): 2,
 ('2019-03-19', 'Fionna and Cake'): 2,
 ('2018-12-11', 'Fionna and Cake'): 1,
 ('2018-11-11', 'Fionna and Cake'): 2,
 ('2018-10-12', 'Fionna and Cake'): 1,
 ('2018-07-16', 'Fionna and Cake'): 1,
 ('2018-06-08', 'Fionna and Cake'): 1,
 ('2018-06-07', 'Fionna a

In [76]:
with open("FD_Adventure_Time_edits_by_day.tsv", "w", encoding='utf-8') as output_file:
    # write a header with three columns
    print("title\tdate\tedits", file=output_file)

    # iterate through every day and title and print out data into the file
    for key in edits_by_day.keys():
        title = key[1]
        day_string = key[0]
        print("\t".join([title, day_string, str(edits_by_day[key])]), file=output_file)

https://docs.google.com/spreadsheets/d/15qARmWTKV0RzANCggy4LieT5oH-8HUEHpB4y9-pTSjQ/edit?usp=sharing

Finally, choose either your Wikipedia or Fandom datasets as the data source for a visualization that shows how each of those articles have grown in length (as measured in characters or "bytes") over time. (Hint: you'll need to return "size" as one of the revision properties (rvprop) if you are not doing it already.)